In [2]:
import os
import xml.etree.ElementTree as ET
from PIL import Image

In [3]:
# Dictionnaire des classes
class_mapping = {"apple": 0, "banana": 1, "orange": 2}

In [4]:
# Dossiers d'entrée et de sortie
xml_dir = 'train_zip/train'  # Le dossier où sont stockés les fichiers .xml
image_dir = 'train_zip/train'  # Le dossier des images correspondantes
output_dir = 'yolo_labels/train'  # Le dossier de sortie pour les fichiers .txt YOLO

os.makedirs(output_dir, exist_ok=True)

In [5]:
# Fonction pour normaliser les coordonnées
def normalize_bbox(xmin, ymin, xmax, ymax, width, height):
    if width == 0 or height == 0:
        raise ValueError(f"Dimensions invalides : largeur={width}, hauteur={height}")
    
    x_center = (xmin + xmax) / 2.0 / width
    y_center = (ymin + ymax) / 2.0 / height
    bbox_width = (xmax - xmin) / width
    bbox_height = (ymax - ymin) / height
    return x_center, y_center, bbox_width, bbox_height

# Convertir chaque fichier XML
for xml_file in os.listdir(xml_dir):
    if xml_file.endswith('.xml'):
        xml_path = os.path.join(xml_dir, xml_file)
        image_path = os.path.join(image_dir, xml_file.replace('.xml', '.jpg'))
        
        print(f"Vérification du fichier XML : {xml_file}")

        # Charger l'arbre XML
        tree = ET.parse(xml_path)
        root = tree.getroot()

        # Récupérer la taille de l'image depuis le fichier XML
        size = root.find('size')
        width = 0
        height = 0
        
        if size is not None:
            width = int(size.find('width').text) if size.find('width') is not None else 0
            height = int(size.find('height').text) if size.find('height') is not None else 0

        if width == 0 or height == 0:
            # Essayer de récupérer la taille de l'image depuis le fichier image
            if os.path.exists(image_path):
                try:
                    with Image.open(image_path) as img:
                        width, height = img.size  # Taille réelle de l'image
                    print(f"Dimensions extraites de l'image {image_path} : largeur={width}, hauteur={height}")
                except Exception as e:
                    print(f"⚠️ Impossible de charger l'image {image_path}: {e}")
                    continue  # Si l'image ne peut pas être ouverte, passer à la suivante
            else:
                print(f"⚠️ L'image correspondante est manquante pour {xml_file}")
                continue  # Si l'image n'existe pas, ignorer ce fichier XML

        # Vérifier si les dimensions sont valides
        if width == 0 or height == 0:
            print(f"⚠️ Dimensions invalides ou manquantes pour {xml_file}")
            continue  # Passer à l'image suivante

        # Créer un fichier .txt pour chaque image
        txt_filename = os.path.splitext(xml_file)[0] + '.txt'
        with open(os.path.join(output_dir, txt_filename), 'w') as txt_file:
            # Parcourir les objets dans le fichier XML
            for obj in root.findall('object'):
                class_name = obj.find('name').text
                if class_name in class_mapping:
                    class_id = class_mapping[class_name]
                    bndbox = obj.find('bndbox')
                    xmin = int(bndbox.find('xmin').text)
                    ymin = int(bndbox.find('ymin').text)
                    xmax = int(bndbox.find('xmax').text)
                    ymax = int(bndbox.find('ymax').text)

                    # Normaliser les coordonnées et écrire dans le fichier .txt
                    try:
                        x_center, y_center, bbox_width, bbox_height = normalize_bbox(xmin, ymin, xmax, ymax, width, height)
                        txt_file.write(f"{class_id} {x_center} {y_center} {bbox_width} {bbox_height}\n")
                    except ValueError as e:
                        print(f"⚠️ Erreur de normalisation pour {xml_file}: {e}")
                        continue  # Passer à l'objet suivant

print("✅ Conversion terminée ! Les fichiers YOLO sont dans le dossier 'yolo_labels/train'")

Vérification du fichier XML : apple_1.xml
Dimensions extraites de l'image train_zip/train\apple_1.jpg : largeur=349, hauteur=349
Vérification du fichier XML : apple_10.xml
Vérification du fichier XML : apple_11.xml
Vérification du fichier XML : apple_12.xml
Vérification du fichier XML : apple_13.xml
Vérification du fichier XML : apple_14.xml
Vérification du fichier XML : apple_15.xml
Vérification du fichier XML : apple_16.xml
Vérification du fichier XML : apple_17.xml
Dimensions extraites de l'image train_zip/train\apple_17.jpg : largeur=700, hauteur=800
Vérification du fichier XML : apple_18.xml
Vérification du fichier XML : apple_19.xml
Vérification du fichier XML : apple_2.xml
Vérification du fichier XML : apple_20.xml
Dimensions extraites de l'image train_zip/train\apple_20.jpg : largeur=500, hauteur=500
Vérification du fichier XML : apple_21.xml
Vérification du fichier XML : apple_22.xml
Vérification du fichier XML : apple_23.xml
Vérification du fichier XML : apple_24.xml
Vérifica